# TP2 WANG Mingxuan

A noter que pour compiler Q1.py en utilisant plusieurs process, il faut taper: `mpirun -np 4 python3 Q1.py`.

# Q1_1. Améliorer la répartition statique des tâches
Dans notre code précédent, on utilisait une distribution en blocs de lignes, c’est-à-dire que chaque processus s’occupait d’un bloc de lignes consécutives. 
Le problème, c’est que la complexité du calcul pour l’ensemble de Mandelbrot n’est pas uniforme :
Les points à l’extérieur de l’ensemble convergent rapidement.
Les points près des bords demandent beaucoup plus d’itérations, donc plus de calcul.

Du coup, on a un souci :
1. Les premiers processus mettent beaucoup plus de temps à finir leur calcul, alors que les derniers finissent très vite. Donc il y a une mauvaise équilibre de charge. 
2. En plus, le temps d’exécution total est déterminé par le processus le plus lent, ce qui réduit l’efficacité du parallélisme.
# Q1_2. Optimisation. Répartition en blocs entrelacés
Pour mieux répartir la charge, on peut utiliser une distribution en blocs entrelacés. Au lieu de donner un bloc entier à chaque processus, on répartit les lignes en alternance.
Avec size = 4, par exemple :

Processus 0 calcule les lignes 0, 4, 8, 12, ...
Processus 1 pour 1, 5, 9, 13, ...
Processus 2 pour 2, 6, 10, 14, ...
Processus 3 pour 3, 7, 11, 15, ... etc...
En fin, on a une meilleure répartition de la charge : les lignes les plus complexes sont séparées entre tous les processus. Et chaque processus a une charge de travail plus homogène, ce qui améliore l’efficacité du calcul parallèle.

Pourtant, il existe également des iinconvénient pour le 2ème méthode.
1. Du coup, quand on utilise Gatherv pour rassembler les résultats, il faut récupérer des données non consécutives depuis plusieurs processus, ce qui complexifie la communication.
2. Et comme l'entrelacement de la figure ne donne pas des groupes de complexité presque égaux, donc il existe encore des problèmes que le temps de calcul de chaque process ne sont pas égaux.
# Q1_3. Maître Esclaves
Le processus maître (Master, rank 0) est responsable de la répartition des tâches et de la collecte des résultats une fois le calcul terminé.

Les processus esclaves (Slaves, rank 1 à size-1) exécutent les calculs de l'ensemble de Mandelbrot et demandent de nouvelles tâches une fois une partie du calcul terminée.

Avantages:
1. Meilleur équilibre de charge :
Chaque processus travaille à son propre rythme, et les tâches sont attribuées en fonction de la disponibilité, réduisant ainsi les temps d'attente inutiles.
2. Adapté aux calculs non uniformes :
La complexité du calcul varie en fonction des zones de l'ensemble de Mandelbrot. L'attribution dynamique des tâches évite que certains processus travaillent plus longtemps pendant que d'autres restent inactifs.

Voici les résultats:

(Pour nbp = 1, unité : second)

|      |  Q1_1| Q1_2 |            
|------|------|------|              
| Process 0 | 1.56 | 1.53 | 

(Pour nbp = 2, unité : second)

|      |  Q1_1| Q1_2 | Q1_3|
|------|------|------|-----|           
| Process 1 | 0.82 | 0.79 | 1.54 |      
| Process 0 | 0.82 | 0.80 | |      
| SpeedUp   | 1.81 | 1.75 | 1.30 |

(Pour nbp = 4, unité : second)

|      |  Q1_1| Q1_2 | Q1_3|
|------|------|------|-----|
| Process 1 | 0.33 | 0.47 | 0.60  |
| Process 2 | 0.32 | 0.47 | 0.52 |
| Process 3 | 0.62 | 0.48 | 0.54 |
| Process 0 | 0.61 | 0.48 |  |
| SpeedUp   | 2.29 | 2.71 | 2.56 |

(Pour nbp = 8, unité : second)

|      |  Q1_1| Q1_2 | Q1_3|
|------|------|------|-----|
| Process 1 | 0.26 | 0.31 | 0.27 |
| Process 2 | 0.35 | 0.20 | 0.26 |
| Process 3 | 0.45 | 0.31 | 0.28 |
| Process 4 | 0.42 | 0.30 | 0.29 |
| Process 5 | 0.36 | 0.30 | 0.31 |
| Process 6 | 0.26 | 0.30 | 0.29 |
| Process 7 | 0.18 | 0.30 | 0.27 |
| Process 0 | 0.20 | 0.30 |  |
| SpeedUp   | 3.41 | 3.96 | 4.01 |



# Q2_1

On a:
$$N_{\text{loc}} = \frac{N}{\text{size}}$$


Pour dimension = 1024:
|      | temps (ms) | speedup |
|------|------|-------|
| n=1 | 0.349 |   1   |
| n=2 | 0.256 | 1.363 |
| n=4 | 0.214 | 1.631 |
| n=8 | 0.165 | 2.115 |

# Q2_2

Pour dimension = 1024:
|      | temps (ms) | speedup |
|------|------|-------|
| n=1 | 0.376 |   1   |
| n=2 | 0.258 | 1.457 |
| n=4 | 0.209 | 1.799 |
| n=8 | 0.222 | 1.694 |

# Q3_1
La loi d'Amdahl :

$$
S(n) = \frac{1}{(1 - P) + \frac{P}{n}}
$$

où :
- S(n) est l’accélération obtenue avec n processeurs,
- P  est celle qui est parallélisable,

On a:
$$
P = 0.9
$$

L’accélération maximale est obtenue donc:

$$
S_{\max} = \lim_{n \to \infty} \frac{1}{(1 - P) + \frac{P}{n}}
$$

On en déduit:

$$
S_{\max} = \frac{1}{1 - P} = \frac{1}{0.1} = 10
$$

Pour conclure, Alice ne pourra jamais obtenir une accélération supérieure à 10 si elle a infiniment de processeurs.


# Q3_2
D’après la loi d’Amdahl, l’accélération croît de manière décroissante lorsque n augmente. 
Calculons d'abord quelques valeurs de n:

$$
S(n) = \frac{1}{(1 - 0.9) + \frac{0.9}{n}}
$$

Pour n = 2, 4, 8, 16 :

$$
S(2) = \frac{1}{0.1 + \frac{0.9}{2}} = \frac{1}{0.1 + 0.45} = \frac{1}{0.55} \approx 1.82
$$

$$
S(4) = \frac{1}{0.1 + \frac{0.9}{4}} = \frac{1}{0.1 + 0.225} = \frac{1}{0.325} \approx 3.08
$$

$$
S(8) = \frac{1}{0.1 + \frac{0.9}{8}} = \frac{1}{0.1 + 0.1125} = \frac{1}{0.2125} \approx 4.71
$$

$$
S(16) = \frac{1}{0.1 + \frac{0.9}{16}} = \frac{1}{0.1 + 0.05625} = \frac{1}{0.15625} \approx 6.40
$$

On observe que :

D'abord, le speedup augmente vite.

Pourtant, à partir de 8 neouds, on multiple le nombre de noeuds pour n'avoir que 1.7 d'augmentation de speedup. Ce qui est un gaspillage de ressource.

Donc pour moi, 8 noeuds c'est bon.



## Q3_3
La loi de Gustafson est:

$$
S(n) = n - (1 - P) \cdot (n - 1)
$$

Si Alice constate une accélération de 4 avec 4 noeuds :

$$
S(4) = 4
$$

Si elle double la quantité de données, et suppose que la complexité du programme est linéaire, alors:

Pour n = 4:

$$
S(4) = 4 - (1 - 0.9) \cdot (4 - 1) = 4 - 0.1 \cdot 3 = 4 - 0.3 = 3.7
$$

Pour n = 8:

$$
S(8) = 8 - (1 - 0.9) \cdot (8 - 1) = 8 - 0.1 \times 7 = 8 - 0.7 = 7.3
$$

Conclusion :
- Si Alice utilise 8 noeuds, elle peut espérer une accélération de 7,3.
- On peut trouver que plus n augmente, plus la valeur est grand, et c'est presque linéaire.

